##  L2S 完整 PyTorch 实现
1. 数据集定义：存储 $h_{X,L'}$ 和 $z_{X,L^*}$
核心是构建「输入语境向量 $h_{X,L'}$」与「目标引导向量 $z_{X,L^*}$」的配对数据集，对应论文中训练辅助网络的输入-标签对。

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim

class SteeringDataset(Dataset):
    """
    数据集：每个样本是 (h_{X,L'}, z_{X,L^*})
    - h_context: [N, D]  → 输入语境向量 h_{X,L'}（L'层最后一个输入token的隐表示）
    - z_target:  [N, D]  → P2S计算的目标引导向量 z_{X,L^*} = h^+ - h^-
    """
    def __init__(self,h_context,z_target):
        assert h_context.shape == z_target.shape, "输入和标签维度必须一致"
        self.h_context = h_context
        self.z_target = z_target
    
    def __len__(self):
        return self.h_context.size(0)
    
    def __getitem__(self,idx):
        return self.h_context[idx],self.z_target[idx]

## 2. 辅助网络 + 训练循环（实现公式 (8)）
### 2.1 两层 MLP 定义（对应 $g_{\Theta^*}$）
论文明确使用「轻量级两层 MLP」作为辅助网络，实现从 $h_{X,L'}$ 到 $z_{X,L^*}$ 的映射 $g_{\Theta^*}: \mathbb{R}^D \to \mathbb{R}^D$。

In [4]:
class SteeringMLP(nn.Module):
    """
    两层感知机（MLP）：实现论文中的 g_Θ^*
    - 输入：h_{X,L'} (维度 D)
    - 输出：预测的引导向量 \hat{z}_{X,L^*} (维度 D)
    """
    def __init__(self,dim_hidden,dim_mlp=1024):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim_hidden,dim_mlp), # 第一层线性变换
            nn.ReLU(), # 激活函数
            nn.Linear(dim_mlp,dim_hidden) # 第二次映射回原维度
        )
    
    def forward(self,x):
        return self.net(x)
    

### 2.2 训练循环（优化公式 (8)）
论文的损失函数是：
$$\Theta^* = \arg\min_{\Theta} \mathbb{E}_X\left[\left\|z_{X,L^*}-g_{\Theta}(h_{X,L'})\right\|_2^2\right]$$
本质是最小化「预测引导向量」与「真实引导向量」的 MSE 损失，以下是完整训练逻辑：

In [5]:
def train_steering_mlp(dim_hidden=2048,
                       num_samples=10000, # 训练样本数量
                       dim_mlp=1024,
                          batch_size=64,
                        num_epochs=10,
                        lr=1e-3,
                        device='cuda' if torch.cuda.is_available() else 'cpu'
                        ):
    # ==============
    # 1、构造示范数据（真实场景替换为自己的h_context和z_target）
    # ==============
    # 模拟 h_context(h_{X,L'}) 和 z_target(z_{X,L^*})
    h_context = torch.randn(num_samples,dim_hidden)
    # 模拟 z_target 
    W_true = torch.randn(dim_hidden,dim_hidden) # 模拟真实映射
    z_target = h_context @ W_true + 0.01 * torch.randn(num_samples,dim_hidden) # 添加噪声

    # 构造数据集和数据加载器
    dataset = SteeringDataset(h_context,z_target)
    dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

    # ==============
    # 2、初始化模型、优化器和损失函数
    # ==============

    model = SteeringMLP(dim_hidden,dim_mlp).to(device)
    optimizer = optim.Adam(model.parameters(),lr=lr)
    criterion = nn.MSELoss()

    # ==============
    # 3、训练模型
    # ==============
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for h_batch,z_batch in dataloader:
            # 迁移数据
            h_batch = h_batch.to(device)
            z_batch = z_batch.to(device)
            # 前向传播
            # g_{\Theta}(h_{X,L'}) -> 预测引导向量
            z_pred = model(h_batch)

            # 计算损失
            # ||z_target - z_pred||_2^2
            loss = criterion(z_pred,z_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 累计损失
            # 每个batch的损失是MSE的mean，需要乘以batch_size来累加
            # 两者相乘 = 当前batch所有样本的损失总和

            total_loss += loss.item()*h_batch.size(0)

        # 打印每个epoch的平均损失
        avg_loss = total_loss / len(dataset)
        print(f"Epoch {epoch + 1} / {num_epochs} - MSE Loss:{avg_loss:.6f}")
    return model

# 运行训练（示范）
if __name__ == "__main__":
    steering_net = train_steering_mlp()

Epoch 1 / 10 - MSE Loss:2021.661381
Epoch 2 / 10 - MSE Loss:1799.521512
Epoch 3 / 10 - MSE Loss:1447.611357
Epoch 4 / 10 - MSE Loss:1190.400927
Epoch 5 / 10 - MSE Loss:1018.923987
Epoch 6 / 10 - MSE Loss:897.652940
Epoch 7 / 10 - MSE Loss:807.108763
Epoch 8 / 10 - MSE Loss:736.789648
Epoch 9 / 10 - MSE Loss:680.595114
Epoch 10 / 10 - MSE Loss:634.770861
